In [ ]:
from cleansing import get_integrated_db, read_table_from_db

def all_data_frames(fancy_db):
    days = read_table_from_db(fancy_db, "days")
    delays = read_table_from_db(fancy_db, "delays")
    snapshots = read_table_from_db(fancy_db, "snapshot")
    candidate_trains = read_table_from_db(fancy_db, "candidate_trains")
    weather = read_table_from_db(fancy_db, "weather")
    return days, delays, snapshots, candidate_trains, weather

days, delays, snapshots, candidate_trains, weather = get_integrated_db(all_data_frames)

In [ ]:
import datetime
import numpy as np
import pandas as pd

weather["date"] = pd.to_datetime(weather["at_time"]).apply(lambda row: row.date())
weather["time"] = pd.to_datetime(weather["at_time"]).apply(lambda row: row.time())
days.day = pd.to_datetime(days.day)

def on_point(hour, df):
    weather_on_point = weather[weather["time"] == datetime.time(hour, 0)].drop(columns=["at_time", "id", "time"])
    weather_on_point["date"] = pd.to_datetime(weather_on_point["date"])
    df.day = pd.to_datetime(df.day)
    df = pd.merge(left=df, right=weather_on_point, left_on='day', right_on='date')
    fancy_string = str(hour) + "00"
    if len(fancy_string) == 3:
        fancy_string = "0" + fancy_string
    df = df.drop(columns=["date"]).rename(columns={
        "temperature": "temp-" + fancy_string,
        "condition": "cond-" + fancy_string
    })
    return df

def changed(from_hour, to_hour, df):
    w_from_hour = weather[weather["time"] == datetime.time(from_hour, 0)].drop(columns=["at_time", "id", "time"])
    w_from_hour["date"] = pd.to_datetime(w_from_hour["date"])
    w_to_hour = weather[weather["time"] == datetime.time(to_hour, 0)].drop(columns=["at_time", "id", "time"])
    w_to_hour["date"] = pd.to_datetime(w_to_hour["date"])
    col_string = "{}-{}".format(from_hour, to_hour)
    
    def do_something(row):
        first = w_from_hour[w_from_hour.date == row["day"]].iloc[0].condition
        second = w_to_hour[w_to_hour.date == row["day"]].iloc[0].condition
        return first == second
    
    df[colstring] = df.apply(do_something)
    return df

days = days.drop(columns=["delay", "vehicle"])
for i in range(0, 9):
    days = on_point(i, days)


In [ ]:
# delays
mean_last_seen = pd.to_datetime(days["last_seen"]).mean()
min_last_seen = pd.to_datetime(days["last_seen"]).min()
mean_first_seen = pd.to_datetime(days["first_seen"]).mean()
min_first_seen = pd.to_datetime(days["first_seen"]).min()

print("mean 1st_seen:\t {}".format(mean_first_seen.time()))
print("mean last_seen:\t {}".format(mean_last_seen.time()))

def calc_first_delay(row):
    row_val = pd.to_datetime(row["first_seen"])
    if row_val > mean_first_seen:
        return row_val - mean_first_seen
    else:
        return - (mean_first_seen - row_val)
    
def calc_last_delay(row):
    row_val = pd.to_datetime(row["last_seen"])
    if row_val > mean_last_seen:
        return row_val - mean_last_seen
    else:
        return - (mean_last_seen - row_val)
    
def calc_abs_first(row):
    row_val = pd.to_datetime(row["first_seen"])   
    if row_val > min_first_seen:
        return row_val - min_first_seen
    else:
        return - (min_first_seen - row_val)
    
def calc_abs_last(row):
    row_val = pd.to_datetime(row["last_seen"])
    if row_val > min_last_seen:
        return row_val - min_last_seen
    else:
        return - (min_last_seen - row_val)
    
    
days["mean_first_delay"] = days.apply(calc_first_delay, axis=1).astype('timedelta64[m]')
days["mean_last_delay"] = days.apply(calc_last_delay, axis=1).astype('timedelta64[m]') - 1
days["abs_first_delay"] = days.apply(calc_abs_first, axis=1).astype('timedelta64[m]')
days["abs_last_delay"] = days.apply(calc_abs_last, axis=1).astype('timedelta64[m]')


In [ ]:
def columns_starting_with(string):
    return days[["day", "first_seen", "last_seen"] + [col for col in list(days.columns) if col.startswith(string)]]

days_conditions = columns_starting_with("cond")
days_temperaturs = columns_starting_with("temp")

In [ ]:
days

In [ ]:
days_conditions

In [ ]:
days_temperaturs

In [ ]:
days.to_csv("days.csv", sep=";", decimal=",")